In [2]:
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
import pandas as pd
import numpy as np
import haversine as hs

### 3. Enter case study information

For running a treatment train from the input sheet (```treatment_train_setup.csv```) enter the following information in this cell:

1. ```case_study```: name of case study to be run
2. ```scenario```: scenario for case study to be run 
3. ```desired_recovery```: default is 1 (for desired recovery of 100%), can be any number between 0-1
3. ```ro_bounds```: default is 'seawater', any other value will give brackish bounds


In [3]:
case_study = 'big_spring'
scenario = 'baseline'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

### 4. Setup WaterTAP3 Model

In [4]:
m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)



Case Study = BIG SPRING
Scenario = BASELINE


=========================ADDING UNIT PROCESSES=========================
WELL FIELD
MICROFILTRATION
REVERSE OSMOSIS
UV AOP
TREATED STORAGE
MUNICIPAL DRINKING
SURFACE DISCHARGE



In [5]:
# Read in brackish database. It is filtered for the correct columns. 
df = pd.read_csv('/Users/amiara/AMO_WW/Brackish/USGS_brackish_filtered.csv', index_col='unique_site_ID')

# convert USGS unit to our units: gpm to m3s, 
df.well_yield = df.well_yield * 6.30902e-5 #gpm to m3s
df.TDS_mgL = df.TDS_mgL / 1000 #mgL to kg/m3
df = df.rename(columns={'TDS_mgL': 'TDS_kgm3'})

# add state look up for elec prices and add elec price column
state_abbv_df = pd.read_csv('../../../../state_abbv.csv', index_col='abbv')
elec_price_df = pd.read_csv('data/electricity_costs.csv', index_col='location')
df['state'] = df.state_alpha.map(state_abbv_df.us_state)
df['elec_price'] = df.state.map(elec_price_df.cost)

# filter for well of interest in the US
df = df[df.well_yield >= 0.01]
df = df[df.TDS_kgm3 >= 0.5]
df = df[df.Latitude >= 22]
df = df[df.Longitude >= -150]
df = df[df.TDS_kgm3 <= 25]

In [6]:
len(df)

5119

In [7]:
# run the baseline watertap3 version of this treatment train
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

Case Study = BIG SPRING
Scenario = BASELINE


=========================ADDING UNIT PROCESSES=========================
WELL FIELD
MICROFILTRATION
REVERSE OSMOSIS
UV AOP
TREATED STORAGE
MUNICIPAL DRINKING
SURFACE DISCHARGE

.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: BASELINE
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/

In [8]:
# unfix so that the membrane area and pressure can be optimized based on source water conditions
m.fs.reverse_osmosis.membrane_area.unfix()
m.fs.reverse_osmosis.feed.pressure.unfix()

# allow TDS levels to be qua

In [9]:
# loop through brackish sites and calculate+save LCOW, capital, O&M, energy intensity, and recovery in a df.
capital = []
recovery = []
onm = []
lcow = []
elec_int = []
tds_out = []
flow_in_list = []
tds_in_list = []
i = 0

for site in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(i/len(df.index))
           
    m.fs.big_spring_feed.flow_vol_in.fix(df.loc[site].well_yield)
    m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(df.loc[site].TDS_kgm3)
    m.fs.well_field.lift_height.fix(df.loc[site].well_depth_ft)
    
    m.fs.costing_param.electricity_price = df.loc[site].elec_price

    m = run_and_return_model(m, objective=True)
    
    flow_in_list.append(df.loc[site].well_yield)
    capital.append(m.fs.costing.capital_investment_total())
    recovery.append(m.fs.costing.system_recovery()*100)
    onm.append(m.fs.costing.operating_cost_annual())
    lcow.append(m.fs.costing.LCOW())
    elec_int.append(m.fs.costing.electricity_intensity())
    tds_out.append(m.fs.municipal_drinking.conc_mass_in[0, 'tds']())
    tds_in_list.append(df.loc[site].TDS_kgm3)
    
df['lcow'] = lcow
df['recovery'] = recovery
df['capital'] = capital
df['onm'] = onm
df['elec_int'] = elec_int
df['tds_out'] = tds_out



-----------------------------
SITE NAME: AZ_GEOTH-AZ0183
0.0001953506544246923
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0274
0.0003907013088493846
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0297
0.000586051963274077
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0327
0.0007814026176987692
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0338
0.0009767532721234617
..

.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-322300N0965030.1
0.007227974213713616
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-325259N1134114.1
0.007423324868138308
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-325600N1092600.1
0.007618675522563001
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-325816N1132449.1
0.007814026176987694
.................................

Degrees of Freedom: 2

In


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-394900N1081000.1
0.014065247118577847
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-395200N0805000.1
0.01426059777300254
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-395300N0830500.1
0.014455948427427232
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-395800N0830200.1
0.014651299081851924
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-451830N1072200.1
0.02090252002344208
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-453235N1224910.1
0.02109787067786677
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-454630N0993702.1
0.021293221332291463
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: FETH-455630N0845422.1
0.021488571986716155
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

..

.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: IL_EPA-ilWL01004
0.027935143582731
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: IL_EPA-ilWL01050
0.028130494237155693
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: IL_EPA-ilWL01122
0.028325844891580385
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: IL_EPA-ilWL01137
0.02852119554600508
.................................

Degrees of Freedom: 2

Initial solve attempt OPTI

...


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-445033098093401
0.9714788044539949
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-445150098123801
0.9716741551084196
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-445240098311501
0.9718695057628443
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-445240098313101
0.972064856417269
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.............


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-454616099372101
0.9785114280132838
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-454958098302301
0.9787067786677085
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-455004100042202
0.9789021293221333
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-455111120462902
0.9790974799765579
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

............


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-462413098203101
0.9855440515725727
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-462413098211702
0.9857394022269974
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-462622097405001
0.9859347528814222
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-462720117001001
0.9861301035358468
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

............


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-472216102444601
0.9925766751318617
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-472453105350301
0.9927720257862864
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-472614098000003
0.9929673764407111
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: USGS-473056122173001
0.9931627270951358
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

............


Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: UT_GEOTH-SL-051
0.9996092986911506
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: UT_GEOTH-SL-052
0.9998046493455753
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: UT_GEOTH-TO-028
1.0
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................


In [ ]:
#df.loc[site]

In [10]:
df.head()

,Unnamed: 0,state_alpha,county_nm,Latitude,Longitude,well_depth_ft,well_yield,TDS_kgm3,state,elec_price,lcow,recovery,capital,onm,elec_int,tds_out
unique_site_ID,,,,,,,,,,,,,,,,
AZ_GEOTH-AZ0183,344,AZ,Pinal,32.639722,-111.945833,682.0,0.031545,0.754442,Arizona,0.0628,0.870494,91.598615,7.750988,0.289007,2.531099,0.038148
AZ_GEOTH-AZ0274,365,AZ,Pinal,32.850556,-111.584444,374.0,0.070667,2.841712,Arizona,0.0628,0.762472,87.431236,13.842740,0.585144,2.531111,0.073151
AZ_GEOTH-AZ0297,371,AZ,Pinal,32.894444,-111.558611,3265.0,0.125000,2.888000,Arizona,0.0628,0.911424,88.237217,21.809374,1.751484,5.942602,0.081833
AZ_GEOTH-AZ0327,375,AZ,Pinal,32.931111,-111.550000,2566.0,0.037854,9.120000,Arizona,0.0628,1.150464,83.476051,9.105903,0.554097,6.098778,0.167580
AZ_GEOTH-AZ0338,376,AZ,Pinal,32.936944,-111.566944,3000.0,0.063000,1.901000,Arizona,0.0628,0.946139,89.695986,12.706416,0.859497,5.434049,0.066118


In [11]:
# for a given brackish site, record all power plants within a certain km radius
def get_nearby_power_plants(df_in, site_in, radius):

    # read in power plant dataset
    netl_df = pd.read_csv("/Users/amiara/GMLC/PowerPlantAnalysis/NETL_ThermalPlants_Filtered.csv")
    # filter for power plants that we have data for, and that only use freshwater
    pp_df = netl_df[((netl_df.WaterType == 'Fresh') & (netl_df.WAvg != '...') & (netl_df.WAvg != '---'))]
    pp_df.WAvg = pp_df.WAvg.astype(float)
    pp_df.rename({'Longitude': 'PPLongitude', 'Latitude': 'PPLatitude'}, axis=1, inplace=True)
    
    dist_km_list = []
    pp_df_out = pd.DataFrame()
    pp_df_out_final = pd.DataFrame()

    bw_lat = df_in.loc[site_in].Latitude
    bw_lon = df_in.loc[site_in].Longitude
    loc1 = (bw_lat, bw_lon)

    for pp_index in pp_df.index: #[1]:#

        pp_lat = pp_df.loc[pp_index].PPLatitude
        pp_lon = pp_df.loc[pp_index].PPLongitude
        loc2 = (pp_lat, pp_lon)

        dist_km = hs.haversine(loc1,loc2)

        if dist_km < radius:

            dist_km_list.append(dist_km)

            pp_df_hold = pp_df[["PlantCode", "Fuel", "Cooling", "Capacity", "PPLatitude", "PPLongitude", "WAvg"]][pp_df.index == pp_index] 
                                #, "Fuel", "Cooling, "Capacity", "Latitude", "Longitude", 
           #'withd', 'cons', 'mup', 'final_with']][pp_df.index == pp_index]

            pp_df_out = pd.concat([pp_df_out,pp_df_hold], ignore_index=True)

    pp_df_out['dist_km'] = dist_km_list
    pp_df_out['site'] = np.array(site_in)

    for c_name in df.columns:
        pp_df_out[c_name] = pp_df_out.site.map(df[c_name])

        # calculate LCOW w/transportation costs

    pp_df_out_final = pd.concat([pp_df_out_final,pp_df_out])

    return pp_df_out_final
    

In [12]:
# loop through all brackish sites, and match power plants. 
df_pp_bw_linked = pd.DataFrame()

i = 0 

for site_in in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(100*i/len(df))
    
    df_site = get_nearby_power_plants(df, site_in, 50) 
    df_pp_bw_linked = pd.concat([df_pp_bw_linked,df_site])

# calculate the cooling demand met by brackish sites (one-to-one match onnly, but should calculate many to one)    
df_pp_bw_linked['withm3s'] = (df_pp_bw_linked.WAvg * 0.0037854 / 3600) * df_pp_bw_linked.Capacity
df_pp_bw_linked['metdemand'] = ((df_pp_bw_linked.recovery/100) * df_pp_bw_linked.well_yield) / df_pp_bw_linked.withm3s


-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.019535065442469232
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.039070130884938464
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.058605196327407696
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.07814026176987693
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.09767532721234616
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.11721039265481539
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.13674545809728464
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.15628052353975386
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.1758155889822231
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.19535065442469232
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.21488571986716157
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.23442078530963079
-----------------------------
SITE NAME: UT_GEOTH-TO-028
0.253955850752100

-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.1488571986716156
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.1683922641140847
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.1879273295565542
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.2074623949990233
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.2269974604414924
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.246532525883962
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.266067591326431
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.2856026567689
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.3051377222113696
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.3246727876538387
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.3442078530963077
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.3637429185387773
-----------------------------
SITE NAME: UT_GEOTH-TO-028
2.3832779839812464
-----------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.278179331900762
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.297714397343231
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.317249462785701
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.336784528228169
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.356319593670639
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.3758546591131084
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.395389724555577
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.414924789998047
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.434459855440516
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.453994920882985
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.473529986325454
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.493065051767924
-----------------------------
SITE NAME: UT_GEOTH-TO-028
4.5126001172103924
-----------------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.427036530572377
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.446571596014847
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.4661066614573155
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.485641726899785
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.5051767923422545
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.524711857784723
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.544246923227193
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.563781988669662
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.583317054112131
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.6028521195546
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.62238718499707
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.6419222504395385
-----------------------------
SITE NAME: UT_GEOTH-TO-028
6.661457315882008
-------------------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.575893729243994
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.595428794686462
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.614963860128931
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.634498925571402
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.65403399101387
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.673569056456339
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.69310412189881
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.712639187341278
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.732174252783746
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.751709318226217
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.771244383668686
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.790779449111154
-----------------------------
SITE NAME: UT_GEOTH-TO-028
8.810314514553625
---------------------------

...

-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.23246727876538
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.25200234420785
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.27153740965032
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.29107247509279
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.31060754053526
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.33014260597773
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.3496776714202
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.36921273686266
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.38874780230513
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.4082828677476
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.42781793319007
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.44735299863254
-----------------------------
SITE NAME: UT_GEOTH-TO-028
90.46688806407502
---------------------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.42039460832194
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.43992967376441
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.45946473920688
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.47899980464935
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.49853487009182
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.51806993553429
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.53760500097675
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.55714006641922
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.57667513186169
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.59621019730416
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.61574526274663
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.6352803281891
-----------------------------
SITE NAME: UT_GEOTH-TO-028
92.65481539363157
--------------------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.58878687243602
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.60832193787849
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.62785700332095
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.64739206876342
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.6669271342059
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.68646219964837
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.70599726509084
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.72553233053331
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.74506739597578
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.76460246141825
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.78413752686072
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.80367259230319
-----------------------------
SITE NAME: UT_GEOTH-TO-028
94.82320765774566
--------------------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.73764407110764
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.7571791365501
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.77671420199258
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.79624926743504
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.81578433287751
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.83531939831998
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.85485446376245
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.87438952920492
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.89392459464739
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.91345966008986
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.93299472553232
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.9525297909748
-----------------------------
SITE NAME: UT_GEOTH-TO-028
96.97206485641726
---------------------------

-----------------------------
SITE NAME: UT_GEOTH-TO-028
98.90603633522173
-----------------------------
SITE NAME: UT_GEOTH-TO-028
98.9255714006642
-----------------------------
SITE NAME: UT_GEOTH-TO-028
98.94510646610667
-----------------------------
SITE NAME: UT_GEOTH-TO-028
98.96464153154913
-----------------------------
SITE NAME: UT_GEOTH-TO-028
98.9841765969916
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.00371166243407
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.02324672787654
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.04278179331901
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.06231685876148
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.08185192420395
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.10138698964641
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.12092205508888
-----------------------------
SITE NAME: UT_GEOTH-TO-028
99.14045712053135
---------------------------

In [13]:
df_pp_bw_linked.head()

,PlantCode,Fuel,Cooling,Capacity,PPLatitude,PPLongitude,WAvg,dist_km,site,Unnamed: 0,...,state,elec_price,lcow,recovery,capital,onm,elec_int,tds_out,withm3s,metdemand
0,55129.0,NGCC,RI,587.0,32.9042,-111.789,265.5114,32.861530,AZ_GEOTH-AZ0183,344,...,Arizona,0.0628,0.870494,91.598615,7.750988,0.289007,2.531099,0.038148,0.163882,0.176315
0,55129.0,NGCC,RI,587.0,32.9042,-111.789,265.5114,20.012172,AZ_GEOTH-AZ0274,365,...,Arizona,0.0628,0.762472,87.431236,13.842740,0.585144,2.531111,0.073151,0.163882,0.377008
0,55129.0,NGCC,RI,587.0,32.9042,-111.789,265.5114,21.536973,AZ_GEOTH-AZ0297,371,...,Arizona,0.0628,0.911424,88.237217,21.809374,1.751484,5.942602,0.081833,0.163882,0.673025
0,8068.0,NGCC,RI,277.0,33.3325,-111.750,272.7277,48.361982,AZ_GEOTH-AZ0327,375,...,Arizona,0.0628,1.150464,83.476051,9.105903,0.554097,6.098778,0.167580,0.079436,0.397793
1,8068.0,NGCC,RI,582.0,33.3325,-111.750,265.6704,48.361982,AZ_GEOTH-AZ0327,375,...,Arizona,0.0628,1.150464,83.476051,9.105903,0.554097,6.098778,0.167580,0.162583,0.194357


In [14]:
df_pp_bw_linked.to_csv('df_pp_bw_linked.csv')

In [15]:
df_pp_bw_linked = pd.read_csv('df_pp_bw_linked.csv')

In [16]:
df_pp_bw_linked[df_pp_bw_linked.dist_km <= 25].to_csv('df_pp_bw_linked<25.csv')
df_pp_bw_linked[df_pp_bw_linked.dist_km > 25].to_csv('df_pp_bw_linked25_50.csv')

In [ ]:
# create code for many to one, and one to many, by using groupby function.

In [ ]:
# scenarios using data from other sub tasks

In [ ]:
# brine disposal costs - could add to WT3 or do after based on brine/recovery factor with TDS at outlet

In [ ]:
# transportation costs - could add to WT3 or do after based on distance.